In [1]:
# -- Import necessary libraries
# from pyimagesearch.centroidtracker import CentroidTracker
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import dlib
import imutils
import time
import cv2 as cv

In [2]:
# Load serialized model Neural Network
net = cv.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

In [3]:
# initialize the video stream and allow the camera sensor to warmup
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
(H, W) = (None, None)
# start the frames per second throughput estimator (instanciate)
fps = FPS().start()

[INFO] starting video stream...


In [4]:
# loop over the frames from the video stream
while True:
    # read the next frame from the video stream and resize it
    frame = vs.read()
    frame = imutils.resize(frame, width=500)
    rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB) # dlib needs RGB image format.
    
    # if the frame dimensions are None, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2] 
        
    # construct a blob from the frame, pass it through the network,
    # obtain our output predictions, and initialize the list of
    # bounding box rectangles
    blob = cv.dnn.blobFromImage(frame, 1.0, (int(W), int(H)), 104, 177, 123)
    net.setInput(blob)
    detections = net.forward()
    rects = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
    # filter out weak detections by ensuring the predicted
    # probability is greater than a minimum threshold
        if detections[0, 0, i, 2] > 0.6:
            # compute the (x, y)-coordinates of the bounding box for
            # the object, then update the bounding box rectangles list
            box = detections[0, 0, i, 3:7] * np.array([W, H, W, H]) # bounding box which coinais the detect object coordinates.
            rects.append(box.astype("int"))

            # draw a bounding box surrounding the object so we can
            # visualize it
            (startX, startY, endX, endY) = box.astype("int")
            cv.rectangle(frame, (startX, startY), (endX, endY),(0, 255, 0), 2)
            cv.putText(frame, "Probabilidade: {:.2f}".format(detections[0, 0, i, 2]), (startX - 10, startY - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv.line(frame, (int(W/2), 0), (int(W/2), H), (0, 0, 255), 2)     
    
    cv.putText(frame, "Entradas: {}".format(str(0)), (10, 50),
                cv.FONT_HERSHEY_SIMPLEX, 0.5, (250, 0, 1), 2)
    cv.putText(frame, "Saidas: {}".format(str(0)), (10, 70),
                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
   # show the output frame
    cv.imshow("Frame", frame)
    key = cv.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

    fps.update()
    
# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
        
# do a bit of cleanup
cv.destroyAllWindows()
vs.stop()

[INFO] elapsed time: 3.25
[INFO] approx. FPS: 7.38
